# This file will evaluate students' homeworks using AI assistant

## 1. Introduce AI tool first

In [12]:
# If use ollama, no API_KEY needed
%load_ext jupyter_ai_magics
%config AiMagics.default_language_model = 'ollama:phi4'

The jupyter_ai_magics extension is already loaded. To reload it, use:
  %reload_ext jupyter_ai_magics


In [ ]:
# If use Gemini, have to load .env file to get API_KEY
%load_ext jupyter_ai_magics
%config AiMagics.default_language_model = 'gemini:gemini-2.0-flash-thinking-exp'

from dotenv import load_dotenv

load_dotenv()

## 2. Use AI to evaluate the given code 

### 2.1 Define the review parametrers

In [42]:
keyword = '----Answer----'
root = './homework'
active = 'lesson1'

print(file_name.strip().split('.')[0].split('_')[-1])

lesson1


In [43]:
import nbformat, os
from nbclient import NotebookClient

class HomeworkManager:
    def __init__(self, key_word, root_dir, active_lesson):
        self.key_word = key_word
        self.root_dir = root_dir
        self.active_lesson = active_lesson
        
    def get_all_homework(self):
        all_homeworks = []
        for dir_path, _, file_names in os.walk(self.root_dir):
            for file_name in file_names:
                # 忽略以点开头的隐藏文件
                if not file_name.startswith('.'): 
                    if file_name.strip().split('.')[0].split('_')[-1] == self.active_lesson:
                        file_path = os.path.join(dir_path, file_name)
                        # print(file_path)
                        all_homeworks.append(file_path)
                    else:
                        print(f"There's no homework for lesson {self.active_lesson}!")
        return all_homeworks
    
    def get_cell_content(self, file_path):
        for dir_path, _, file_names in os.walk(self.root_dir):
            for file_name in file_names:
                file_path = os.path.join(dir_path, file_name)
                all_homeworks.append(file_path)
        return all_homeworks
    
    def get_cell_content(self, file_path):
        with open(file_path) as f:
            nb = nbformat.read(f, as_version=4)
        review_cell = [cell for cell in nb.cells if self.key_word.lower() in cell.source.lower()]
        if len(review_cell) == 0:
            raise IndexError("There's no answer in the notebook.")
        else:
            return review_cell

In [44]:
import time
from IPython.core.getipython import get_ipython

class NotebookManager:
    def __init__(self):
        self.shell = get_ipython()

    def create_code_cell(self, content, cell_type='code'):
        """Create new code cell"""
        payload = dict(
            source = 'set_next_input',
            text = content,
            replace = False
        )
        self.shell.payload_manager.write_payload(payload)

    def create_markdown_cell(self, content, cell_type='markdown'):
        """Create new markdown cell"""
        payload = dict(
            source = 'set_next_input',
            text = content,
            replace = False
        )
        self.shell.payload_manager.write_payload(payload)

    def run_cell(self, code):
        result = self.shell.run_cell(code)
        return result
    
    def create_and_run(self, content):
        self.create_code_cell(content)
        time.sleep(0.5)
        return self.run_cell(content)

In [45]:
hws = HomeworkManager(keyword, root, active)

homeworks = hws.get_all_homework()

nb = NotebookManager()

for homework in homeworks:
    file_name = os.path.basename(homework)
    stu_name = file_name.split('_')[0]
    code_seg = hws.get_cell_content(homework)[0].source
    # print(hw_dict['name'])
    prompt = "%%ai\n"
    prompt += "{stu_name} is a beginner in Python coding. Please review his(her) code shown in {code_seg}, rated them in 5-score system and give some advice from the perspective of {stu_name}'s teacher, omit advanced coding skills."
    prompt += "Use Chinese to format your answer. When you finish reviewing, add a seperator line and write a EOF marker."

    nb.create_and_run(prompt)


<think>
好，我需要评估Yula的Python代码，并根据老师的角度给出评分和建议。用户希望评分为5分制，避免涉及高级技能，使用中文输出，并在末尾添加分割线和EOF标记。

首先，阅读Yula的代码：

1. 建立航天员档案：变量命名清晰，打印语句正确。
2. 发射报告部分包括发射时间、天气状况，以及多行状态检查。这部分格式化正确。
3. 倒计时使用了while循环和time.sleep，逻辑基本正确。

评分指标：
1. 变量命名与代码清晰度
2. 字符串格式化与输出
3. 控制结构与逻辑
4. 代码结构与注释
5. 多行打印与格式

逐一评估：

1. **变量命名与代码清晰度**：Yula使用了有意义的变量名，如commander_name、mission_time等。但建议统一使用英文变量名以提高国际化。

2. **字符串格式化与输出**：正确使用f-strings，无错误，评分5/5。

3. **控制结构与逻辑**：倒计时逻辑正确，但循环条件是`countdown > 0`，无法显示0。建议改为`>= 0`。

4. **代码结构与注释**：代码分部分清晰，但缺少注释，影响可读性。评分3/5。

5. **多行打印与格式**：使用三引号处理多行字符串有效，但内容较多时可拆分为简单print语句。评分4/5。

总结优点和改进建议：

- 代码结构清晰，完成度高。
- 添加注释以提高可读性。
- 修改倒计时条件显示到0。
- 统一使用英文变量名。

最后，用Markdown格式输出结果，并添加分割线和EOF标记。
</think>

```markdown
### 代码评分与建议  

Yula 的代码整体逻辑清晰，完成度较高！以下是基于老师角度的评分和建议：  

---

### **1. 变量命名与代码清晰度**  
**评分：4/5**  
- 优点：变量命名清晰且具有实际意义（如 `commander_name`、`mission_time`），能够很好地反映变量的作用。  
- 改进建议：可以尝试将变量名统一为英文，以提高代码的国际化程度和可读性。  

---

### **2. 字符串格式化与输出**  
**评分：5/5**  
- 优点：使用 `f-string` 格式化字符串操作得当，且内容打印清晰。  
- 改进建议：无明显改进建议，已达到初级编程要求。  

---

### **3. 控制结构与逻辑**  
**评分：4/5**  
- 优点：倒计时逻辑完整，使用 `while` 循环和 `time.sleep(1)` 实现了基本的计时功能。  
- 改进建议：可以考虑将倒计时的结束条件改为 `countdown >= 0`，以便在循环结束后打印 `0` 的情况，更加完整。  

---

### **4. 代码结构与注释**  
**评分：3/5**  
- 优点：整体结构清晰，能够分步骤完成任务（如航天员档案、发射报告、倒计时）。  
- 改进建议：在关键步骤前添加简单注释，帮助他人更好地理解代码逻辑。例如：  
  ```python
  # 多行打印系统状态检查
  print('''
  系统状态检查：
  1. 燃料系统 - 正常
  2. 导航系统 - 正常
  3. 生命支持 - 正常
  已进入深海状态
  ''')
  ```

---

### **5. 多行打印与格式**  
**评分：4/5**  
- 优点：使用三引号实现多行字符串，避免了多次调用 `print()` 的繁琐操作。  
- 改进建议：如果内容较多或需要调整格式，可以尝试拆分为多个简单的 `print()` 语句，方便后续维护和修改。

---

### **总结与建议**  
Yula 的代码整体完成度较高，逻辑清晰！以下是一些改进建议：  
1. 在关键步骤前添加注释，帮助他人更好地理解代码逻辑。  
2. 如果希望倒计时显示到 `0`，请将循环条件改为 `countdown >= 0`。  
3. 统一使用英文变量名（如 `launch_time`），提高代码的国际化程度。  

希望你继续努力，写出更加优雅和高效的 Python 代码！  

---  

**EOF**

<think>
好的，我现在需要评估Ethan的Python代码，并从老师的角度给出评分和建议。用户要求使用5分制评分，忽略高级技能，用中文输出，并在末尾添加分割线和EOF标记。

首先，我仔细阅读了Ethan的代码：

1. **建立航天员档案**：
   - 使用`import time`导入模块。
   - 定义了三个变量：`commander_name`, `mission_time`, `rocket_type`，并赋值。
   - 打印这些变量，并用`=`乘以50作为分隔线。

2. **发射报告**：
   - 打印“发射控制系统已启动”。
   - 定义了`launch_time`和`weather`两个变量，并分别打印。

3. **多行打印系统状态检查**：
   - 使用三引号实现多行字符串，打印系统状态。
   - 打印“自然选择，前进四!”。
   - 打印一个换行符`\n`。

4. **发射倒计时**：
   - 初始化`countdown`为10。
   - 使用`while countdown > 0`循环，每次打印当前倒计时并递减。
   - 每次循环后暂停1秒（使用`time.sleep(1)`）。
   - 循环结束后，打印“Launch!!!”。

接下来，我需要按照用户的要求进行评分和建议。评分包括五个方面：变量命名与初始化、控制结构与逻辑、代码格式与注释、多行字符串使用以及输出结果。

**变量命名与初始化（满分5分）**：
Ethan的变量命名清晰，如`commander_name`、`mission_time`等，易于理解。所有变量都正确初始化，并且类型合适。因此，我给满分5分。

**控制结构与逻辑（满分5分）**：
代码使用了`while`循环进行倒计时，逻辑正确，能够完成预期任务。虽然可以考虑改进结束条件以显示0，但目前的逻辑已经满足要求。所以，我也给满分5分。

**代码格式与注释（满分5分）**：
Ethan在代码中添加了一些注释，帮助理解不同部分的功能，这很好。然而，有些地方可以更详细，比如在关键步骤前添加更多解释。此外，代码缩进正确，但可以进一步优化排版。因此，我给4.5分。

**多行字符串使用（满分5分）**：
Ethan成功使用三引号实现了多行打印，避免了多次调用`print()`函数的繁琐。这是一个很好的实践，所以我给满分5分。

**输出结果准确性（满分5分）**：
所有打印语句都准确无误地显示了预期内容。倒计时功能正常，并且在每次循环后暂停1秒，符合预期。因此，我给满分5分。

接下来是总结与建议部分：

Ethan的代码整体完成度较高，逻辑清晰，变量命名合理，并正确使用了多行字符串和控制结构。然而，可以进一步优化注释，增加更多解释性说明，帮助他人理解代码逻辑。此外，在倒计时结束后显示0会更完整。建议继续学习Python基础知识，提升代码质量。

最后，按照用户的要求，用中文输出结果，并在末尾添加分割线和EOF标记。
</think>

```markdown
# 代码评审报告 - Ethan 的 Python 代码

## 评分标准（满分5分）

### 1. **变量命名与初始化**
**得分：5/5**

- **优点**：
  - 变量命名清晰，如 `commander_name`、`mission_time` 等。
  - 所有变量都正确初始化，并且类型合适。

### 2. **控制结构与逻辑**
**得分：5/5**

- **优点**：
  - 使用了 `while` 循环进行倒计时，逻辑正确，能够完成预期任务。
  - 倒计时功能正常，每次循环后暂停1秒，符合要求。

### 3. **代码格式与注释**
**得分：4.5/5**

- **优点**：
  - 在关键部分添加了注释，如 `# 发射报告` 和 `# 多行打印`。
  - 代码缩进正确，易于阅读。

- **建议改进**：
  - 可以在关键步骤前添加更多解释性注释，帮助他人更好地理解代码逻辑。
  - 进一步优化代码排版，例如在变量和函数之间增加空白行。

### 4. **多行字符串使用**
**得分：5/5**

- **优点**：
  - 成功使用三引号实现了多行打印，避免了多次调用 `print()` 的繁琐。
  - 多行字符串格式正确，内容清晰易读。

### 5. **输出结果准确性**
**得分：5/5**

- **优点**：
  - 所有打印语句都准确无误地显示了预期内容。
  - 倒计时功能正常，并且在每次循环后暂停1秒，符合预期。

---

## 总结与建议

Ethan 的代码整体完成度较高，逻辑清晰，变量命名合理，并正确使用了多行字符串和控制结构。以下是几点改进建议：

1. **注释优化**：
   - 在关键步骤前添加更多解释性注释，帮助他人更好地理解代码逻辑。

2. **倒计时改进**：
   - 可以考虑在倒计时结束后显示0，使倒计时更完整。

3. **继续学习**：
   - 继续学习 Python 的基础知识，例如函数、列表和字典的使用，提升代码质量。

---

# 评分总结

| 项目                  | 得分 |
|-----------------------|------|
| 变量命名与初始化      | 5    |
| 控制结构与逻辑        | 5    |
| 代码格式与注释        | 4.5  |
| 多行字符串使用        | 5    |
| 输出结果准确性        | 5    |
| **总得分**            | **24.5/25** |

---

# 评审人建议

Ethan 的代码整体表现优秀，但在注释和格式方面还有提升空间。继续努力，保持良好的编程习惯！

---

```